In [1]:
from pathlib import Path
import re
import pandas as pd

In [2]:
# каталог в котором находятся подкаталоги с информацией об образах
reportdir = Path(r"D:\Работа\2023\38\исследование бэкапов")

In [3]:
reportdir

WindowsPath('D:/Работа/2023/38/исследование бэкапов')

In [4]:
def revert_string(s):
    """
    функции передается строки из файла с хэшами файлов
    а возвращается словарик {'key': хэш-сумма, 'value': путь к файлу в формате str}
    """
    a = s.rstrip().split('\t')
    return {'key':a[2], 'value':a[0]}

In [5]:
def hash_dict(bigdata, image_name, filename):
    """
    bigdata: глобальный список, в который должны записываться все файлы
    image_name: имя образа - ключ для словаря, указывающий, в каком образе лежат конкретные файлы
    filename: файл сос писком контрольных сумм, который подвергается обработке
    """
    with open(filename, "r", encoding = 'utf-8-sig') as f:
        for  i in f: # бежим по строкам
            row = revert_string(i)
            hash_= row['key']
            path_ = row['value']
            record = [hash_, image_name, path_]
            bigdata.append(record)
        


In [6]:
"""
сюда будут писаться все хэши
словарь представляет такую штуку: основной ключ - это хэш-сумма
в качестве значения сожержится словарь, у которого в качестве ключа имя каталога, в котором лежит списк файлов с хэшами, 
а в качестве значения словарь, в котором перечислены все файлы, с данным хэщем из данного образа
{хэш файла : { номер образа : [сисок файлов с данным хэшем на образе]}}
"""
bigdata = []

In [7]:
for i in reportdir.glob('*'): # бежим по папкам
    print(i) # выводим название папки
    # чтоб не делать слишком много работы, сужаем зону поиска (первые 10 каталогов)
    dir_proc = re.findall('.*(\d{2}) OK', str(i)) 
#     dir_proc = re.findall('.*(0\d) OK', str(i)) 
    if dir_proc:
        image_name = dir_proc[0] # получаем номер образа, чтоб использовать как ключ
        mdf = i.joinpath("md5_sha1_list.csv") # путь к файлу с хэш-суммами
        if mdf.exists():
            print(mdf)
            hash_dict(bigdata, image_name, mdf)


D:\Работа\2023\38\исследование бэкапов\.ipynb_checkpoints
D:\Работа\2023\38\исследование бэкапов\00 OK
D:\Работа\2023\38\исследование бэкапов\00 OK\md5_sha1_list.csv
D:\Работа\2023\38\исследование бэкапов\01 OK
D:\Работа\2023\38\исследование бэкапов\01 OK\md5_sha1_list.csv
D:\Работа\2023\38\исследование бэкапов\02 OK
D:\Работа\2023\38\исследование бэкапов\02 OK\md5_sha1_list.csv
D:\Работа\2023\38\исследование бэкапов\03 OK
D:\Работа\2023\38\исследование бэкапов\03 OK\md5_sha1_list.csv
D:\Работа\2023\38\исследование бэкапов\04 OK
D:\Работа\2023\38\исследование бэкапов\04 OK\md5_sha1_list.csv
D:\Работа\2023\38\исследование бэкапов\05 OK
D:\Работа\2023\38\исследование бэкапов\05 OK\md5_sha1_list.csv
D:\Работа\2023\38\исследование бэкапов\06 OK
D:\Работа\2023\38\исследование бэкапов\06 OK\md5_sha1_list.csv
D:\Работа\2023\38\исследование бэкапов\07 OK
D:\Работа\2023\38\исследование бэкапов\07 OK\md5_sha1_list.csv
D:\Работа\2023\38\исследование бэкапов\08 OK
D:\Работа\2023\38\исследование бэ

## Начинается нормальная работа
Здесь создается плоский список о всеми файлами, содержащимися в образах. 
Файл выглядит следующим образом:<br>
контрольная сумма файла;   номер образа, в котором содержится файл;    имя файла<br>
В итоге от имен файлов мы избавимся, так они не нужны для определения образов в которых содержится наибольшее количество файлов с растпространенными контрольными суммами



In [9]:
a = pd.DataFrame(bigdata, columns = ['hash', 'img', 'filepath'])

In [11]:
a.shape

(7165431, 3)

In [14]:
a.to_csv("файлы со всех образов с хэшами.csv", sep='\t', index=False)

In [15]:
len(a['hash'].unique())

1380849

In [16]:
a_sample = a.copy()

In [17]:
# сгруппировали файлы по хэшам и нашли самые распространенные хэши. Очевидно, что это пустые файлы
# sample_groups = a_sample.groupby('hash')
# sample_groups.count()["img"].sort_values()

In [18]:
# в общем определеили хэш нулевого файла. Все эти файлы нужно удалить
zero_hash = "da39a3ee5e6b4b0d3255bfef95601890afd80709"

In [19]:
a_sample.shape

(7165431, 3)

In [20]:
# просто для проверки, что это зафайлы. Вывожу в список, потому что по-другому пандас их обрезает
# list(a_sample[a_sample["hash"]==zero_hash]["filepath"].head(30))

In [21]:
# удаляем строки, в которых содержатся файлы нулевой диниы
a_sample.drop(a_sample[a_sample["hash"] ==zero_hash].index, inplace = True)

In [22]:
a_sample.shape

(6948860, 3)

In [23]:
# эта операция считает, в каоих образах хэш-сумма повторяется(без подсчета количества повторений в одном образе)
# так как нас интересует в какая хэш сумма в каких образах содержится, чтобы выбрать самый популярный образ
# уникальные файлы - какой хэш в скольки образах встречается
prisut = a_sample.groupby('hash')['img'].unique().apply(len).rename("in_images").reset_index()
prisut

,hash,in_images
0,-,10
1,-------------------,22
2,00000c47e6dc9bdd54a53e90619fcccd36b49cb5,4
3,00001f0c29dd6609baec384da65072895185dcfb,1
4,000023a909db614ae9086871664302adf809c56a,1
...,...,...
1380843,ffffe2518d0bbbc6087d9345b03efdebc74bf6f8,2
1380844,ffffe647692ab7dd96dcda28e38aadad429eae84,1
1380845,ffffe94036cc94ea9e7d2221da02412821384ff2,1
1380846,ffffeff9c81708a6b64c652e8a1bc865950f5bdf,5


In [24]:
# слепляем две таблицы - со всеми файлам и с хэш-сумами. Слияние происходит по хэш-суммам
a_sample = a_sample.merge(prisut)

In [25]:
a_sample.shape

(6948860, 4)

In [26]:
# убираем имена файлов, чтобы они не спамили названия и не создавали слишком большие суммы
# на интересуют контрольные суммы и в каких образах они встречаются. 
# Сколько раз одн контрольная сумма встречается в одном образе это не должно быть особо интересно
a_sample.drop('filepath', axis =1, inplace = True)


In [27]:
# теперь нужно удалить  повторяющиеся строки
a_sample.drop_duplicates(subset=["hash", "img"], keep = "first", inplace = True)
len(a_sample)

4744650

In [28]:
# оставляем в выборке только файлы, которые встречаюстя в нескольких образах (не уникальные)
a_sample = a_sample.loc[a_sample["in_images"]>1,:]
len(a_sample)

4202667

In [29]:
# считаю количество неуникальных хэшей в каждом образе
# далее беру индекс образа с максимальным количеством хэшей
# в качестве индекса серии используется название образа, поэтому в дальнейшем легко фильтровать образы
# я фильтрую общий датафрейм по максимальному образу и получаю все хэши из этого образа (серия)
ch = a_sample.groupby("img")['hash'].count()
chi = ch.idxmax()
to_del = a_sample.loc[a_sample["img"] == chi, 'hash']
to_del

883                             -------------------
3010       2df502a944ff721241be20a9e449d2acd07e0312
4065       82b615b86a4be01cee7815525806fb4102cdddc8
4084       278a2e1e930444c3b7b74257b75579b81b03f697
4103       4ad3f7574c39dae1f33de2371cb98269eea6eee0
                             ...                   
5948417    7a7d1aa5ee757f8ab931a8ff0fdf59cfeb905baf
5948422    4b88b92d30308c5aa0b710c1634600741898901d
5948426    bb9a4fee357a491da1682dbc181497b0cff1f1c9
5948431    c1268b416dd7d2115f52b2d2425ca6072f4ebeb1
5948435    395a09d1d498c0a906a2e0e568e18242fe706323
Name: hash, Length: 311301, dtype: object

In [30]:
ch

img
00     74323
01    138539
02    138486
03     86919
04     87624
05    191447
06    100179
07    198779
08    203375
09    203849
10    249435
11    253032
12    294097
13    311301
14    289157
15     15796
16     15833
17     15857
18     19583
19     19582
20     15778
21    156442
22     15780
23    223722
24    222388
25    131397
26    221193
27    132229
28    131411
29     43586
30      1548
Name: hash, dtype: int64

In [31]:
# из серии можно взять только значения, чтобы индексы не мешались
# эти значения мы будем удалять из других образов
to_del.values

array(['-------------------', '2df502a944ff721241be20a9e449d2acd07e0312',
       '82b615b86a4be01cee7815525806fb4102cdddc8', ...,
       'bb9a4fee357a491da1682dbc181497b0cff1f1c9',
       'c1268b416dd7d2115f52b2d2425ca6072f4ebeb1',
       '395a09d1d498c0a906a2e0e568e18242fe706323'], dtype=object)

In [32]:
# удаление из датафрейма всех строк с хэшами, присутствующими в избранном образе
# a_sample.drop(a_sample[a_sample['hash'].isin(vug.values)].index, inplace = True)

In [33]:
a_sample2 = a_sample.copy()

In [34]:
a_sample = a_sample2.copy()

In [35]:
imgs_count = len(a_sample.groupby("img").groups.keys())
imgs_count

31

### Здесь мы делаем много работы, но посути единственная польза: это определение очередности, с которой образы будут вычитаться из общего списка

In [36]:
def remove_img(ig):
        tc = ig['hash'].count().sort_values()
        max_image_index = tc.idxmax()
        # img_seq.append(max_image_index)
        print ("следующий образ в цепочке копирования", max_image_index)
        to_del = a_sample.loc[a_sample["img"] == max_image_index, 'hash']
        vug = to_del.values
        a_sample.drop(a_sample[a_sample['hash'].isin(vug)].index, inplace = True)
        return max_image_index


In [37]:
a_sample = a_sample2.copy()
img_seq = []
# функция итеративно удаляет контрольные суммы, содержащиеся всамом большом образе из других образов
# пока что не реализовано создание списков уникальных файлов
# да и вообще. файлы, не встречающиеся в других образах просто вырезаны, чтоб не мешать предыдущим экспериментам
gk = None
ig = a_sample.groupby("img") # датафрейм сргуппированный по образам
old_gk = ig.groups.keys()
imgs_count = len(a_sample.groupby("img").groups.keys())
counter =0
while True:
    print(f'||{counter}||----------------------------------------------------------')
    print(a_sample['img'].value_counts())
    if len(old_gk)<1:
        break
    img_ind = remove_img(ig)
    print(f"УДАЛЯЕМЫЙ ОБРАЗ: {img_ind}")
    ig = a_sample.groupby("img")
    gk = ig.groups.keys()
    print("old:", old_gk)
    print("new:", gk)
    print("длина пула: ", len(gk))
    if si:=(old_gk - gk):
            print(f'{si=}')
            print(type(img_seq), img_seq)
            if {img_ind} != si:
                si -= {img_ind}
                img_seq.append(img_ind)
                print(f"отдельно добавляем номер удаляемого образа {img_ind}")
            print(f"добаляем множество {si}")    
            img_seq.extend(list(si))
    old_gk = gk
    counter +=1
img_seq.extend(list(old_gk))



||0||----------------------------------------------------------
img
13    311301
12    294097
14    289157
11    253032
10    249435
23    223722
24    222388
26    221193
09    203849
08    203375
07    198779
05    191447
21    156442
01    138539
02    138486
27    132229
28    131411
25    131397
06    100179
04     87624
03     86919
00     74323
29     43586
18     19583
19     19582
17     15857
16     15833
15     15796
22     15780
20     15778
30      1548
Name: count, dtype: int64
следующий образ в цепочке копирования 13
УДАЛЯЕМЫЙ ОБРАЗ: 13
old: dict_keys(['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30'])
new: dict_keys(['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30'])
длина пула:  30
si={'13'}
<class 'list'> [

In [38]:
img_seq

['13',
 '09',
 '02',
 '10',
 '27',
 '24',
 '21',
 '17',
 '19',
 '18',
 '07',
 '04',
 '12',
 '06',
 '23',
 '20',
 '22',
 '26',
 '03',
 '29',
 '30',
 '11',
 '08',
 '25',
 '28',
 '01',
 '00',
 '15',
 '16',
 '05',
 '14']

In [39]:
len(img_seq)

31

## Начинаем формировать списки для копирования

In [40]:
a_sample = a.copy()

In [41]:
a_sample.columns

Index(['hash', 'img', 'filepath'], dtype='object')

In [42]:
for img in img_seq:
    print(a_sample['img'].value_counts().sort_index())
    img_frame = a_sample.loc[a_sample['img'] == img]
    hashes = img_frame.loc[:, 'hash'].unique()
    files = img_frame.loc[:, ['filepath', 'hash']].sort_values(by= 'filepath')
    
    # files.to_csv(f"files_img_{img}.csv", encoding='utf-8-sig', sep='\t', index=False, header =False)
    
    files.to_csv(f"files_img_{img}.csv", columns = ['filepath'], encoding='utf-8-sig', sep='\t', index=False, header =False)
    a_sample.drop(a_sample[a_sample['hash'].isin(hashes)].index, inplace = True)    

img
00    123680
01    194829
02    189458
03    147575
04    157218
05    310172
06    190647
07    309897
08    298935
09    329767
10    345367
11    340614
12    511983
13    497937
14    522687
15     47072
16     47257
17     47493
18     35511
19     35503
20     44609
21    405968
22     45290
23    554734
24    347811
25    208767
26    348700
27    206876
28    198012
29    105245
30     15817
Name: count, dtype: int64
img
00    108836
01    172407
02    168568
03    126339
04    135647
05    185003
06     78358
07    182040
08    172014
09    208324
10    120030
11    110664
12    157259
14     75438
15     34754
16     34798
17     34868
18     34083
19     34075
20     33254
21    210000
22     33362
23    147978
24     87516
25    185230
26     89079
27    183373
28    174814
29     88967
30     15667
Name: count, dtype: int64
img
00    108162
01    171752
02    167956
03    125515
04    134516
05     42400
06     46958
07     35624
08      6702
10    101274
11     91767


In [43]:
len(hashes)

56884

In [44]:
files

,filepath,hash
3948089,I:\$Recycle.Bin\S-1-5-21-1288033057-2243439731...,254f650e8f95b39779f845f4d9c981b73a87410c
3948090,I:\$Recycle.Bin\S-1-5-21-1288033057-2243439731...,2b0edc748989a600034c5a6661d94efe5c1b30e7
3948092,I:\$Recycle.Bin\S-1-5-21-1288033057-2243439731...,01de36bde1fd9e9563872fd3b5b6b19c37da2d78
3977654,I:\Program Files (x86)\AOMEI\AOMEI Backupper\7...,303cf91d7d55cc27bbc3288d184236b182f155b5
3977667,I:\Program Files (x86)\AOMEI\AOMEI Backupper\7...,ddd1b7639e06386a74e603a57d6fe7c878a48608
...,...,...
4374734,I:\Windows\servicing\Sessions\Sessions.back.xml,865c00c484189ead84c5774883f0ec8d4f488354
4374735,I:\Windows\servicing\Sessions\Sessions.xml,865c00c484189ead84c5774883f0ec8d4f488354
4167266,I:\Windows\setupact.log,8769f403f273fdaa0f7c881fac1ee1ca049bbcfa
4167269,I:\Windows\splwow64.exe,8681bc7794c4cb5b3d178340152ccfb6877a85b5


In [45]:
len(files)

69268

In [147]:
all_files = []
for i in reportdir.glob('*'): # бежим по папкам
    if i.is_file():
        print(i) # выводим название папки
        dir_proc = re.findall('files_img_(\d{2}).csv', str(i)) 
        if dir_proc:
            dir_proc= dir_proc[0]
            with open(i, "r", encoding = 'utf-8-sig') as f:
                for  r in f: # бежим по строкам
                    fi, ha = r.rstrip().split("\t")
                    all_files.append([ha, dir_proc, fi])

D:\Работа\2023\38\исследование бэкапов\anus.txt
D:\Работа\2023\38\исследование бэкапов\crop_files_img_01_dir_2.csv
D:\Работа\2023\38\исследование бэкапов\crop_pahts_count_files_in_dir.py
D:\Работа\2023\38\исследование бэкапов\files_img_00.csv
D:\Работа\2023\38\исследование бэкапов\files_img_01.csv
D:\Работа\2023\38\исследование бэкапов\files_img_02.csv
D:\Работа\2023\38\исследование бэкапов\files_img_03.csv
D:\Работа\2023\38\исследование бэкапов\files_img_04.csv
D:\Работа\2023\38\исследование бэкапов\files_img_05.csv
D:\Работа\2023\38\исследование бэкапов\files_img_06.csv
D:\Работа\2023\38\исследование бэкапов\files_img_07.csv
D:\Работа\2023\38\исследование бэкапов\files_img_08.csv
D:\Работа\2023\38\исследование бэкапов\files_img_09.csv
D:\Работа\2023\38\исследование бэкапов\files_img_10.csv
D:\Работа\2023\38\исследование бэкапов\files_img_11.csv
D:\Работа\2023\38\исследование бэкапов\files_img_12.csv
D:\Работа\2023\38\исследование бэкапов\files_img_13.csv
D:\Работа\2023\38\исследовани

In [148]:
b = pd.DataFrame(all_files, columns=['hash','img', 'filename'])

In [149]:
b

,hash,img,filename
0,232b4614e793e5e22a1e9f4eb56fef80e4f525fc,00,J:\Program Files (x86)\AIDA64 Business\aida_ic...
1,fc2ccb7f67cc8a4a57b3c4e2336cd9be5ee41a23,00,J:\Program Files (x86)\AIDA64 Business\unins00...
2,8644f5a15be16e488b1b9885b091a2367551af42,00,J:\Program Files (x86)\AOMEI Backupper\AmCore.dll
3,d404a8cb2ff65150e7d8c5214e04aee164cda96e,00,J:\Program Files (x86)\AOMEI Backupper\log\ABC...
4,2723584f4e7dc450c360bcb2e612338130f72051,00,J:\Program Files (x86)\AOMEI Backupper\log\ABC...
...,...,...,...
1799404,5cf1df5ddcf53e0eaa888e4dbbc64d2ea3f6782a,30,I:\РАЗНОЕ\Экзаменационные билеты с альтернатив...
1799405,8ca5dc36fb2c1a374f5fad87741fe97181303900,30,I:\РАЗНОЕ\Энциклопедия судебной практики Наруш...
1799406,f97669bdf0cf7486390f81e4fa1c1a55038f1c1b,30,I:\РАЗНОЕ\список обучение управление АЗС.xlsx
1799407,75774afd72648b44faf46b9676c81b3266ee318e,30,I:\РАЗНОЕ\№ФПК-1000_р от 16.11.2020(1)(2).pdf


In [84]:
a_sample = a.copy()

In [85]:
one_file = pd.DataFrame(columns=['hash','img', 'filepath'])
one_file

,hash,img,filepath


In [86]:
for img in img_seq:
    print(a_sample['img'].value_counts().sort_index())
    img_frame = a_sample.loc[a_sample['img'] == img]
    hashes = img_frame.loc[:, 'hash'].unique()
    files = img_frame.loc[img_frame['hash'].isin(hashes), :].sort_values(by= 'filepath')
    files = files.drop_duplicates(subset= ['hash'])
    one_file = pd.concat([one_file, files], axis=0)
    a_sample.drop(a_sample[a_sample['hash'].isin(hashes)].index, inplace = True)  

img
00    123680
01    194829
02    189458
03    147575
04    157218
05    310172
06    190647
07    309897
08    298935
09    329767
10    345367
11    340614
12    511983
13    497937
14    522687
15     47072
16     47257
17     47493
18     35511
19     35503
20     44609
21    405968
22     45290
23    554734
24    347811
25    208767
26    348700
27    206876
28    198012
29    105245
30     15817
Name: count, dtype: int64
img
00    108836
01    172407
02    168568
03    126339
04    135647
05    185003
06     78358
07    182040
08    172014
09    208324
10    120030
11    110664
12    157259
14     75438
15     34754
16     34798
17     34868
18     34083
19     34075
20     33254
21    210000
22     33362
23    147978
24     87516
25    185230
26     89079
27    183373
28    174814
29     88967
30     15667
Name: count, dtype: int64
img
00    108162
01    171752
02    167956
03    125515
04    134516
05     42400
06     46958
07     35624
08      6702
10    101274
11     91767


In [87]:
one_file.reset_index(inplace = True, drop=True)

In [88]:
one_file

,hash,img,filepath
0,38b98c9520974d622a26e23262467291f35a804c,13,J:\$AV_ASW\$VAULT\1c01eb61a1c18ad030d7407d3369...
1,22f908c7e4fe181942b95b993fe61a55aa309e31,13,J:\$AV_ASW\$VAULT\vault.db
2,2df502a944ff721241be20a9e449d2acd07e0312,13,J:\$Recycle.Bin\S-1-5-18\desktop.ini
3,c87510212cab922d22f29b681ad592345c2da3e5,13,J:\$Recycle.Bin\S-1-5-21-1288033057-2243439731...
4,7db549d8c7c349ed9c1cf62e39f7d35419c2f93a,13,J:\$Recycle.Bin\S-1-5-21-1288033057-2243439731...
...,...,...,...
1380844,b377d32316a82cc9f20737a2ca8e16dd5e4d7545,14,I:\Windows\servicing\Sessions\31015133_2935101...
1380845,22fc5c8641156dc294dcf8f8e9c5f8b4a023f663,14,I:\Windows\servicing\Sessions\31015133_2935101...
1380846,865c00c484189ead84c5774883f0ec8d4f488354,14,I:\Windows\servicing\Sessions\Sessions.back.xml
1380847,8769f403f273fdaa0f7c881fac1ee1ca049bbcfa,14,I:\Windows\setupact.log


In [89]:
len(one_file['hash'].unique())

1380849

In [90]:
one_file.to_csv('файлы из всех образов с уникальными хэшами.csv', sep='\t', index=False)

проверил, сколько уникальных конирольных сумм содержится в итоговом сводном файле

In [77]:
len(a['hash'].unique())

1380849

потом проверил, сколько уникальных контрольных сумм содержится в изначальном файле
### Значения совпали - ни одна контрольная сумма не потерялась

In [78]:
one_file['filename'].unique()

array([nan], dtype=object)

In [80]:
one_file['img'].value_counts().sort_index()

img
00      5284
01     13264
02    135008
03     18814
04     35683
05     29140
06     30362
07     32672
08      4177
09    163960
10     88442
11      3447
12     44396
13    327544
14     56884
15         9
16         5
17     15842
18        14
19     11799
20       256
21     96741
22       185
23     52628
24     61011
25     11315
26      8562
27     72753
28      9412
29     41944
30      9296
Name: count, dtype: int64

In [81]:
a['img'].value_counts().sort_index()

img
00    123680
01    194829
02    189458
03    147575
04    157218
05    310172
06    190647
07    309897
08    298935
09    329767
10    345367
11    340614
12    511983
13    497937
14    522687
15     47072
16     47257
17     47493
18     35511
19     35503
20     44609
21    405968
22     45290
23    554734
24    347811
25    208767
26    348700
27    206876
28    198012
29    105245
30     15817
Name: count, dtype: int64